In [ ]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [ ]:
import pandas as pd
import numpy as np
import math
import re
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import os

## To follow up on the previous model on detecting Covid-19 on Chest X-Rays and trying to come up with other helpful solutions to improve people's lives during these hard times, we will focus on another aspect of this Pandemic - Mental Health. The intent here is to build a question and answer transformer model to answer people's questions in  regards to mental health. Mental Health is another crucial component of overall well being and many people are likely to show symptoms or exacerbate existing symptoms during the pandemic due to periods of paranoia, extended isolation, etc. It would be helpful to have an easily accessible chatbot or question and answer model, possibly through a website interface, that can provide interactive answers to mental health questions from users. The key of this project is to show the potential of a transformer solution for this problem, for which a real solution would have to be vetted by a mental health/medical and data science teams. The project uses the paper "Attention Is All You Need" by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin or ArXiv.org submitted on 12 Jun 2017 (v1), last revised 6 Dec 2017, v5 to create the question and answer model. <br><br>

## The Dataset consists of 98 question and answer pairs and was prepared by https://www.kaggle.com/narendrageek with the following acknowledgements:
* https://www.thekimfoundation.org/faqs/
* https://www.mhanational.org/frequently-asked-questions
* https://www.wellnessinmind.org/frequently-asked-questions/
* https://www.heretohelp.bc.ca/questions-and-answers

In [ ]:
# mount drive

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Copy python file containing utilities from drive

!cp /gdrive/MyDrive/utils.py .

In [ ]:
# import as a module to use classes and functions developed directly from
# the paper "Attention is All You Need" for modeling and for additional 
# processing

import utils

In [ ]:
# Dataset is in a zip file so we will use the appropriate python tools

import zipfile

with zipfile.ZipFile("/gdrive/MyDrive/archive.zip","r") as zip_ref:
   zip_ref.extractall('/gdrive/MyDrive/archive/Mental_Health_FAQ.csv')

In [ ]:
data_path = '/gdrive/MyDrive/archive/Mental_Health_FAQ.csv/'
fileName = 'Mental_Health_FAQ.csv'

In [ ]:
# Use OS library to open

df = pd.read_csv(os.path.join(data_path, fileName), encoding = None)
 

In [ ]:
# Examine first five entries

df.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [ ]:
# Check file types, number of rows, etc.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Question_ID  98 non-null     int64 
 1   Questions    98 non-null     object
 2   Answers      98 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [ ]:
# Take a look at the answers which have more content

df['Answers'][0]

'Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical condition due to the brainâ€™s biology.\nSimilarly to how one would treat diabetes with medication and i

In [ ]:
# There are some encoding errors we are going to need to fix


df['Answers'] = df['Answers'].map(lambda x: x.encode('ascii', errors = 'replace').decode('utf-8'))
df['Answers'][0]

'Mental illnesses are health conditions that disrupt a person???s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a person???s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical condition due to the brain???s biology.\nSimilarly to how one would treat diabetes with medication and i

In [ ]:
# Now that the characters are utf-8, let's correct the errors by replacing the strings

df['Answers'] = df['Answers'].map(lambda x: x.replace('\n', ' '))

In [ ]:
df['Answers'] = df['Answers'].map(lambda x: x.replace("???", "'"))

In [ ]:
df['Answers'] = df['Answers'].map(lambda x: x.replace(" ? s", "'s"))

In [ ]:
df['Answers'][0]

"Mental illnesses are health conditions that disrupt a person's thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life. Mental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital. It is important to know that mental illnesses are medical conditions that have nothing to do with a person's character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical condition due to the brain's biology. Similarly to how one would treat diabetes with medication and insulin, m

In [ ]:
df['Questions'][10]

'How can I find a mental health professional right for my child or myself?'

In [ ]:
# Add start and end tokens to sentences

df['Questions'] = ["<start> " + utils.preprocess_sentence(sentence) + " <end>" for sentence in df['Questions'].values.tolist()]
df['Answers'] = ["<start> " + utils.preprocess_sentence(sentence) + " <end>" for sentence in df['Answers'].values.tolist()]

In [ ]:
df['Questions'][0]

'<start> what does it mean to have a mental illness ? <end>'

In [ ]:
df['Answers'][0]

"<start> mental illnesses are health conditions that disrupt a person's thoughts , emotions , relationships , and daily functioning . they are associated with distress and diminished capacity to engage in the ordinary activities of daily life . mental illnesses fall along a continuum of severity some are fairly mild and only interfere with some aspects of life , such as certain phobias . on the other end of the spectrum lie serious mental illnesses , which result in major functional impairment and interference with daily life . these include such disorders as major depression , schizophrenia , and bipolar disorder , and may require that the person receives care in a hospital . it is important to know that mental illnesses are medical conditions that have nothing to do with a person's character , intelligence , or willpower . just as diabetes is a disorder of the pancreas , mental illness is a medical condition due to the brain's biology . similarly to how one would treat diabetes with 

In [ ]:
df.iloc[95]['Answers']

"<start> you can't . but you can influence their capacity to make good choices in a world where alcohol and other drugs are available . it's about helping your child develop the skills to assess what might be helpful in achieving their goals in life . it's also about nourishing a supportive relationship , so they know where to go with questions or problems . opening up a discussion about drugs can help strengthen your relationship with your child . inviting and allowing open , honest conversation about drugs or any other subject makes your child know that what they are thinking , feeling and experiencing matters to you . the goal is to get your child talking and sharing their thoughts and feelings . when young people are asked thoughtful , open ended questions exploring what they think , it helps them become interested in their own thoughts and behaviour . this process of self reflection is part of developing critical thinking skills , a part of good decision making . critical thinking

In [ ]:
df.head()

,Question_ID,Questions,Answers
0,1590140,<start> what does it mean to have a mental ill...,<start> mental illnesses are health conditions...
1,2110618,<start> who does mental illness affect ? <end>,<start> it is estimated that mental illness af...
2,6361820,<start> what causes mental illness ? <end>,<start> it is estimated that mental illness af...
3,9434130,<start> what are some of the warning signs of ...,<start> symptoms of mental health disorders va...
4,7657263,<start> can people with mental illness recover...,"<start> when healing from mental illness , ear..."


In [ ]:
# save data for later use
df_attention = df.to_csv('/gdrive/My Drive/df_attention.csv', header=True)

In [ ]:
# The processed data set answers section is too long. The transformer model 
# could probably handle it if we had more memory or if we distributed the data
# set but the purpose of this project is to understand the transformer model and
# to get an application working with the resources that we have on hand
# this is why we will reduce the answers section and make sure that the answer 
# answers the question, meaning that we need to manually extract the context
# from the processed answer by using the sentences that make the most sense if 
# the answer does not make contextual sense we will change the question or change
# the answer so that it does make sense and if this is not possible simply 
# eliminate the question. This is being done because with the current length of 
# the sequences the RAM is rapidly overwhelmed, so we have to essentially create
# a custom dataset

df = pd.read_csv('/gdrive/MyDrive/df_attention_edited_final.csv')

In [ ]:
questions = df['Questions'].values.tolist()

In [ ]:
# We will tokenize all sentences (convert to numbers so that we can use them for prediction)

tokenizer_questions = tf.keras.preprocessing.text.Tokenizer(num_words=None, filters='', # list of characters
                                                  lower=True)                 # to filter is empty
tokenizer_questions.fit_on_texts(questions)                                                # string

questions_sequence = tokenizer_questions.texts_to_sequences(questions)

In [ ]:
len(tokenizer_questions.word_counts)

276

In [ ]:
questions_sequence[10]

[1, 9, 6, 4, 11, 10, 8, 13, 30, 15, 20, 31, 3, 2]

In [ ]:
len(questions)

158

In [ ]:
VOCAB_SIZE_QUESTIONS = len(tokenizer_questions.word_counts) + 2

In [ ]:
answers = df['Answers'].values.tolist()

In [ ]:
# Tokenizer for answers

tokenizer_answers = tf.keras.preprocessing.text.Tokenizer(num_words=None, filters='', # list of characters
                                                  lower=True)                 # to filter is empty
tokenizer_answers.fit_on_texts(answers)                                                # string

answers_sequence = tokenizer_answers.texts_to_sequences(answers)

In [ ]:
len(tokenizer_answers.word_counts)

1079

In [ ]:
VOCAB_SIZE_ANSWERS = len(tokenizer_answers.word_counts) + 2

In [ ]:
answers_sequence[0]

[7,
 12,
 195,
 21,
 15,
 243,
 22,
 457,
 4,
 74,
 47,
 91,
 2,
 126,
 2,
 127,
 2,
 5,
 111,
 458,
 1,
 8]

In [ ]:
len(answers)

158

In [ ]:
# Create padding so that we keep the sequences at the same length and establish a max length
MAX_LENGTH = 26

questions = tf.keras.preprocessing.sequence.pad_sequences(questions_sequence,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=MAX_LENGTH)
answers = tf.keras.preprocessing.sequence.pad_sequences(answers_sequence,
                                                        value=0,
                                                        padding='post',
                                                        truncating='post',
                                                        maxlen=MAX_LENGTH)

In [ ]:
answers[0]

array([  7,  12, 195,  21,  15, 243,  22, 457,   4,  74,  47,  91,   2,
       126,   2, 127,   2,   5, 111, 458,   1,   8,   0,   0,   0,   0],
      dtype=int32)

In [ ]:
df.iloc[0]['Answers']

'<start> mental illnesses are health conditions that disrupt a person s thoughts , emotions , relationships , and daily functioning . <end>'

In [ ]:
# Create the dataset, batch size and improve accessibility to data during training

BUFFER_SIZE = 1000
BATCH_SIZE = 512
dataset = tf.data.Dataset.from_tensor_slices((questions, answers))
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

### Attention for Mental Health Part 2 - Pre Processing and Modeling

In [ ]:
tf.keras.backend.clear_session()

# Hyper-parameters
D_MODEL = 1024
NB_LAYERS = 4
FFN_UNITS = 2048
NB_PROJ = 8 
DROPOUT_RATE = 0.1

transformer = utils.Transformer(vocab_size_enc=VOCAB_SIZE_QUESTIONS,
                          vocab_size_dec=VOCAB_SIZE_ANSWERS,
                          d_model=D_MODEL,
                          nb_layers=NB_LAYERS,
                          FFN_units=FFN_UNITS,
                          nb_proj=NB_PROJ,
                          dropout_rate=DROPOUT_RATE)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction="none")

def loss_function(target, pred):
    mask = tf.math.logical_not(tf.math.equal(target, 0))
    loss_ = loss_object(target, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)
        
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                     beta_1=0.9,
                                     beta_2=0.98,
                                     epsilon=1e-9)

In [ ]:
EPOCHS = 315
for epoch in range(EPOCHS):
    print("Start of epoch {}".format(epoch+1))
  
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for (batch, (enc_inputs, targets)) in enumerate(dataset):
        dec_inputs = targets[:, :-1]
        dec_outputs_real = targets[:, 1:]
        with tf.GradientTape() as tape:
            predictions = transformer(enc_inputs, dec_inputs, True)
            loss = loss_function(dec_outputs_real, predictions)
        
        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
        
        train_loss(loss)
        train_accuracy(dec_outputs_real, predictions)
        
        if batch % 50 == 0:
            print("Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}".format(
                epoch+1, batch, train_loss.result(), train_accuracy.result()))

Start of epoch 1
Epoch 1 Batch 0 Loss 6.9363 Accuracy 0.0028
Start of epoch 2
Epoch 2 Batch 0 Loss 6.9390 Accuracy 0.0038
Start of epoch 3
Epoch 3 Batch 0 Loss 6.9303 Accuracy 0.0046
Start of epoch 4
Epoch 4 Batch 0 Loss 6.9372 Accuracy 0.0033
Start of epoch 5
Epoch 5 Batch 0 Loss 6.9070 Accuracy 0.0041
Start of epoch 6
Epoch 6 Batch 0 Loss 6.8897 Accuracy 0.0051
Start of epoch 7
Epoch 7 Batch 0 Loss 6.8833 Accuracy 0.0058
Start of epoch 8
Epoch 8 Batch 0 Loss 6.8525 Accuracy 0.0051
Start of epoch 9
Epoch 9 Batch 0 Loss 6.8426 Accuracy 0.0053
Start of epoch 10
Epoch 10 Batch 0 Loss 6.8040 Accuracy 0.0078
Start of epoch 11
Epoch 11 Batch 0 Loss 6.7767 Accuracy 0.0063
Start of epoch 12
Epoch 12 Batch 0 Loss 6.7394 Accuracy 0.0056
Start of epoch 13
Epoch 13 Batch 0 Loss 6.7054 Accuracy 0.0076
Start of epoch 14
Epoch 14 Batch 0 Loss 6.6609 Accuracy 0.0063
Start of epoch 15
Epoch 15 Batch 0 Loss 6.6094 Accuracy 0.0089
Start of epoch 16
Epoch 16 Batch 0 Loss 6.5727 Accuracy 0.0099
Start of e

In [ ]:
def pre_process(tokenizer_questions, sentence, maxlen=26):

    sentence = ["<start> " + utils.preprocess_sentence(sentence) + " <end>"]
    #tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, filters='', # list of characters
                                                  #lower=True)                 # to filter is empty
    #tokenizer.fit_on_texts(sentence)                                                # string

    sentence_sequence = tokenizer_questions.texts_to_sequences(sentence)

    return tf.keras.preprocessing.sequence.pad_sequences(sentence_sequence,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=MAX_LENGTH)

In [ ]:
def evaluate(inp_sentence):

    evaluated = pre_process(tokenizer_questions, inp_sentence)
    inp_sentence = evaluated
    enc_input = inp_sentence
    
    output = tf.expand_dims([VOCAB_SIZE_ANSWERS-2], axis=0)
    
    for _ in range(MAX_LENGTH):
        predictions = transformer(enc_input, output, False)
        
        prediction = predictions[:, -1:, :]
        
        predicted_id = tf.cast(tf.argmax(prediction, axis=-1), tf.int32)
        
        if predicted_id == VOCAB_SIZE_ANSWERS-1:
            return tf.squeeze(output, axis=0)
        
        output = tf.concat([output, predicted_id], axis=-1)
        
    return tf.squeeze(output, axis=0)

In [ ]:
def reply(tokenizer_answers, sentence, max_len=26):
    output = evaluate(sentence)

    lst = output.numpy().tolist()
    
    predicted_sentence = [tokenizer_answers.index_word[i] for i in lst]

    item = ''

    for i in predicted_sentence:

      #if i == '.':
        #break

      if i == '<end>':
        break

      elif (i != '?') and (i != 'carcinogens'):
        item = item + i + ' '
        

    
    print("Input: {}".format(sentence))
    print("Predicted Response: {}".format(item.replace(' s ',"'s ").replace(' m ',"'m ").replace(' t ',"'t ") ))
    return item.replace(' s ',"'s ").replace(' m ',"'m ").replace(' t ',"'t ") #+ '.'

In [ ]:
def preprocess_reference(reference):
    
    reference = reference.split()
    
    item = ''

    for i in reference:

      #if i == '.':
        #break

      if i == '<end>':
        break

      elif (i != '?') and (i != '<start>'):
        item = item + i + ' '
        

    

    return item.replace(' s ',"'s ").replace(' m ',"'m ").replace(' t ',"'t ") # + '.'

In [ ]:
def preprocess_questions(question):
    
    question = question.split()
    
    item = ''

    for i in question:

      if i == '<end>':
        break

      elif (i != '<start>'):
        item = item + i + ' '
        

    

    return item

In [ ]:
import collections

In [ ]:
def compute_f1(gold_toks, pred_toks):
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return 1 # If either is no-answer, then F1 is 1 if they agree, 0 otherwise return int(gold_toks == pred_toks)
        if num_same == 0:
            return 0
    precision = 1.0 * num_same / len (pred_toks)
    recall = 1.0 * num_same / len (gold_toks)
    f1 = (2 * precision * recall) / ((precision + recall) + 0.0001)
    return f1

In [ ]:
df.iloc[0]['Questions']

'<start> what is mental illness ? <end>'

In [ ]:
df.iloc[0]['Answers']

'<start> mental illnesses are health conditions that disrupt a person s thoughts , emotions , relationships , and daily functioning . <end>'

In [ ]:
reference = preprocess_reference(df.iloc[0]['Answers'])
candidate = reply(tokenizer_answers, preprocess_questions(df.iloc[0]['Questions']))

Input: what is mental illness ? 
Predicted Response: mental health conditions that disrupt a person's thoughts , emotions , relationships , and daily functioning . 


In [ ]:
print('reference :', reference[:len(candidate)])
print('candidate :', candidate)

reference : mental illnesses are health conditions that disrupt a person's thoughts , emotions , relationships , and daily 
candidate : mental health conditions that disrupt a person's thoughts , emotions , relationships , and daily functioning . 


F1= 2*precision*recall/(precision+recall)
precision = tp/(tp+fp)
recall=tp/(tp+fn)

precision = 1.0 * num_same / len(pred_toks)=tp/(tp+fp)
recall = 1.0 * num_same / len(gold_toks)=tp/(tp+fn)

tp=number of tokens that are shared between the correct answer and the prediction<br>
fp=number of tokens that are in the prediction but not in the correct answer<br>
fn=number of tokens that are in the correct answer but not in the prediction<br>


https://kierszbaumsamuel.medium.com/f1-score-in-nlp-span-based-qa-task-5b115a5e7d41


In [ ]:
score = compute_f1(reference[:len(candidate)].split(), candidate.split())
score

0.8823029440096434

In [ ]:
scores = []
for i in range(len(df)):
    reference = preprocess_reference(df.iloc[i]['Answers'])
    candidate = reply(tokenizer_answers, preprocess_questions(df.iloc[i]['Questions']))
    score = compute_f1(reference[:len(candidate)].split(), candidate.split())
    scores.append(score)


Input: what is mental illness ? 
Predicted Response: mental health conditions that disrupt a person's thoughts , emotions , relationships , and daily functioning . 
Input: what does mental illness cause ? 
Predicted Response: mental illnesses cause distress and diminished capacity to engage in the ordinary activities of daily life . 
Input: who does mental illness affect ? 
Predicted Response: mental illness can affect anyone regardless of gender , age , income , social status , ethnicity , religion , sexual orientation , or background background 
Input: what causes mental illness ? 
Predicted Response: possible causes of mental illness include genetics, infections, brain defects or injury, prenatal damage, substance abuse and other factors . 
Input: what are some of the warning signs of mental illness ? 
Predicted Response: some of the warning signs of mental illness are confused thinking, long lasting sadness, irritability, extreme highs and lows in mood, excessive fear , worrying , 

In [ ]:
scores

[0.8823029440096434,
 0.9999500024998749,
 0.9614884641383263,
 0.9999500024998749,
 0.9614884641383263,
 0.9056103979167187,
 0.9999500024998749,
 0.7916167566189632,
 0.6414594556561742,
 0.9999500024998749,
 0.9999500024998749,
 0.9999500024998749,
 0.9999500024998749,
 0.9999500024998749,
 0.9999500024998749,
 0.9999500024998749,
 0.9999500024998749,
 0.8888389602806263,
 0.6221722509271479,
 0.6521239168765177,
 0.16211250607921898,
 0.9999500024998749,
 1,
 0.9332833607004746,
 0.8935670467062025,
 0.9999500024998749,
 0.7036537758334497,
 0.6499501288266536,
 0.9999500024998749,
 0.9267793007308976,
 0.9999500024998749,
 0.7499531279295043,
 0.5651673957270335,
 0.4827091608347519,
 0.814764886461503,
 0.9614884641383263,
 0.0,
 0.13328357413232395,
 0.8518019233655225,
 0.4389779941957354,
 0.846103849108217,
 0.9230269257851098,
 0.5454046987705609,
 0.9999500024998749,
 0.3749503537957475,
 0.9056103979167187,
 0.9999500024998749,
 0.9614884641383263,
 0.8845653874413119,
 0.

In [ ]:
# overall average f1 score

np.mean(scores)

0.8117977077263838

In [ ]:
# For unseen samples select a few examples with high f1 scores, we can rephrase the questions to test
# the model

# Unseen example - a question used in training was what is mental illness?
# we will use the same reference answer as the gold standard but will phrase the question differently

new = reply(tokenizer_answers, 'define mental illness?')

Input: define mental illness?
Predicted Response: mental illness are health conditions that disrupt a person's thoughts , emotions , relationships , and daily functioning . 


In [ ]:
reference = preprocess_reference(df.iloc[0]['Answers'])


In [ ]:
score = compute_f1(reference[:len(new)].split(), new.split())
score

0.918868958158396

In [ ]:
# Unseen example - a question used in training was are there local resources?
# we can use the same reference answer as the gold standard but will phrase
# the question differently

new = reply(tokenizer_answers, 'where are there local resources?')

Input: where are there local resources?
Predicted Response: you can learn more about resources in your community by searching online . 


In [ ]:
reference = preprocess_reference(df.iloc[34]['Answers'])
reference

'yes , you can learn more about resources in your community by searching online . '

In [ ]:
# This reference answer is the closest we have in the dataset 
# calculating the f1 score doesn't really give the predicted response justice 
# since as humans we can understand it was a very strong
# reply to that unseen question but we are limited by our methods

score = compute_f1(reference[:len(new)].split(), new.split())
score

0.814764886461503

In [ ]:
# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:

# cumulative BLEU scores

from nltk.translate.bleu_score import sentence_bleu

scores = []
for i in range(len(df)):
    reference = preprocess_reference(df.iloc[i]['Answers'])
    candidate = reply(tokenizer_answers, preprocess_questions(df.iloc[i]['Questions']))

    
    print('Cumulative 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
    print('Cumulative 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0)))
    print('Cumulative 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0)))
    print('Cumulative 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)))

Input: what is mental illness ? 
Predicted Response: mental health conditions that disrupt a person's thoughts , emotions , relationships , and daily functioning . 
Cumulative 1-gram: 0.198198
Cumulative 2-gram: 0.445195
Cumulative 3-gram: 0.586196
Cumulative 4-gram: 0.667229
Input: what does mental illness cause ? 
Predicted Response: mental illnesses cause distress and diminished capacity to engage in the ordinary activities of daily life . 
Cumulative 1-gram: 0.192661
Cumulative 2-gram: 0.438931
Cumulative 3-gram: 0.580740
Cumulative 4-gram: 0.662519
Input: who does mental illness affect ? 
Predicted Response: mental illness can affect anyone regardless of gender , age , income , social status , ethnicity , religion , sexual orientation , or background background 
Cumulative 1-gram: 0.141026
Cumulative 2-gram: 0.375534
Cumulative 3-gram: 0.523924
Cumulative 4-gram: 0.612808
Input: what causes mental illness ? 
Predicted Response: possible causes of mental illness include genetics, i

In [ ]:


scores = []
for i in range(len(df)):
    reference = preprocess_reference(df.iloc[i]['Answers'])
    candidate = reply(tokenizer_answers, preprocess_questions(df.iloc[i]['Questions']))
    score = sentence_bleu(reference, candidate)
    scores.append(score)
    print("bleu score: ", score)

Input: what is mental illness ? 
Predicted Response: mental health conditions that disrupt a person's thoughts , emotions , relationships , and daily functioning . 
bleu score:  0.6672290177922376
Input: what does mental illness cause ? 
Predicted Response: mental illnesses cause distress and diminished capacity to engage in the ordinary activities of daily life . 
bleu score:  0.6625187887633293
Input: who does mental illness affect ? 
Predicted Response: mental illness can affect anyone regardless of gender , age , income , social status , ethnicity , religion , sexual orientation , or background background 
bleu score:  0.6128081331864039
Input: what causes mental illness ? 
Predicted Response: possible causes of mental illness include genetics, infections, brain defects or injury, prenatal damage, substance abuse and other factors . 
bleu score:  0.634395177463988
Input: what are some of the warning signs of mental illness ? 
Predicted Response: some of the warning signs of mental 

In [ ]:
# Overall average bleu score
np.mean(scores)

0.6448033113740166